This notebook demonstrates projection of point cloud at the viewpoint of nearby panorma as rasters, and export clipped rasters.

In [167]:
import numpy as np
# from matplotlib import cm
from matplotlib import pyplot as plt
# from pyproj import Transformer
# from scipy.spatial.transform import Rotation as R
# from lidar.point_cloud_processings import remove_noise, get_rotation_matrix, project_lidar_perspective, project_lidar_equirectangular
import geopandas as gpd
import pdal
import glob
import pyproj
import os
from PIL import Image
from scipy import ndimage
from scipy.ndimage import map_coordinates, generic_filter
from skimage.io import imsave
from skimage.transform import resize

### Input and outputs

In [168]:
building_points_file=r'/home/ubuntu/lavender_floor_height/output/Final_Wagga_training_samples_pano_metadata_clipping.geojson'
las_files_folder = r"/mnt/floorheightvolume/lidar_Wagga/clipped/"
out_folder=r"/mnt/floorheightvolume/lidar_Wagga/clipped_projected/"
os.makedirs(out_folder, exist_ok=True)

### Load building points

In [169]:
gdf_building_points=gpd.read_file(building_points_file)
gdf_building_points=gdf_building_points[gdf_building_points["USAGE"]=="Residential"].reset_index(drop=True)
gdf_building_points.head()

,WALL_M,STEPS,USAGE,STOREYS,ASSESSOR,address,Area_1,PMF,Dep_500,Dep_200,...,LTP_z_m,Roll_deg,Pitch_deg,Heading_deg,FRAMEID,IMGID,distance,house_loc_left,house_loc_right,geometry
0,Brick,1,Residential,1,ROBERT,28 SPRING STREET,128.806974,6.9281,2.1277,-9999.0000,...,180.0966,-0.343451,-1.215830,193.554297,1739405253-0000003052,0504_1739405253-0000003052_Panorama_20250213_G...,18.901973,1506,3950,POINT (147.35145 -35.11277)
1,Brick,2,Residential,1,ROBERT,9 NORTH PARADE,154.755345,6.5745,-9999.0000,-9999.0000,...,180.7244,1.000789,-0.061197,101.069299,1739405253-0000003272,0505_1739405253-0000003272_Panorama_20250213_G...,19.948013,1449,3893,POINT (147.35326 -35.11334)
2,Brick,1,Residential,2,ROBERT,4 THOMAS STREET,294.580219,6.8309,2.1030,1.1167,...,180.1386,0.110114,-0.999721,340.351146,1739332533-0000003367,0448_1739332533-0000003367_Panorama_20250212_G...,21.276532,1504,3949,POINT (147.35074 -35.1112)
3,Brick,1,Residential,1,ROBERT,40 SLOCUM STREET,290.352249,6.2612,1.6366,0.5181,...,180.9586,-0.614706,-0.105462,71.002953,1739332533-0000006081,0274_1739332533-0000006081_Panorama_20250212_G...,12.458696,6990,9434,POINT (147.35142 -35.1056)
4,Brick,1,Residential,1,ROBERT,13 EVANS STREET,205.000441,5.7174,1.1566,-9999.0000,...,180.3083,-0.504672,1.289210,159.715010,1739332533-0000007076,0344_1739332533-0000007076_Panorama_20250212_G...,27.159260,1502,3947,POINT (147.35177 -35.10797)


## Test workflow with one building example

### Identify corresponding las file

In [170]:
# building_ufi=1271
# gdf_building_points[gdf_building_points['UFI']==building_ufi]

In [171]:
i=90
building_ufi=gdf_building_points.iloc[i]['UFI']
house_loc_left, house_loc_right = int(gdf_building_points.iloc[i]['house_loc_left']), int(gdf_building_points.iloc[i]['house_loc_right'])
las_file_path=glob.glob(las_files_folder+'*'+'_UFI_'+str(building_ufi)+'.las')[0]
las_file_path

'/mnt/floorheightvolume/lidar_Wagga/clipped/gnaf_GANSW706136463_UFI_1271.las'

In [172]:
def project_las_to_equirectangular( input_las, camera_pos=[0, 0, 0], camera_angles=[0, 0, 0], 
                                   width=2048, height=1024, nodata_float=9999, nodata_int=255):
    """
    Projects LAS to equirectangular maps with intrinsic XYZ rotation.
    Returns:
        rgb_raster (np.uint8): (H,W,3) RGB image
        z_raster (np.float32): (H,W) elevation map
        depth_raster (np.float32): (H,W) depth map
        class_raster (np.float32): (H,W) classification map
    """
    # --- Data Loading ---
    pipeline = pdal.Reader.las(filename=input_las).pipeline()
    pipeline.execute()
    points = pipeline.arrays[0]
    x, y, z = points["X"], points["Y"], points["Z"]
    rgb = np.vstack([points["Red"], points["Green"], points["Blue"]]).T/256 # 16 bits
    classification = points["Classification"].astype(np.uint8)
    intensity = points["Intensity"].astype(np.uint8)
    print("RGB min/max:", rgb.min(axis=0), rgb.max(axis=0))

    # --- Coordinate Transformation ---
    # Convert angles to radians
    yaw_rad = np.radians(camera_angles[0])
    pitch_rad = np.radians(camera_angles[1])
    roll_rad = np.radians(camera_angles[2])

    # Translate to camera origin
    x -= camera_pos[0]
    y -= camera_pos[1]
    z -= camera_pos[2]

    # Intrinsic XYZ rotation matrices
    R_roll = np.array([
        [1, 0, 0],
        [0, np.cos(roll_rad), -np.sin(roll_rad)],
        [0, np.sin(roll_rad), np.cos(roll_rad)]
    ])
    R_pitch = np.array([
        [np.cos(pitch_rad), 0, np.sin(pitch_rad)],
        [0, 1, 0],
        [-np.sin(pitch_rad), 0, np.cos(pitch_rad)]
    ])
    R_heading = np.array([
        [np.cos(yaw_rad), -np.sin(yaw_rad), 0],
        [np.sin(yaw_rad), np.cos(yaw_rad), 0],
        [0, 0, 1]
    ])
    R_total = R_heading @ R_pitch @ R_roll  # Intrinsic XYZ order

    # Apply rotation
    coords = np.vstack([x, y, z])
    coords_local = R_total @ coords

    # Transform to camera coordinate convention:
    #    LiDAR's +Z (up) should become camera's +Y (down)
    #    LiDAR's +Y (north) should become camera's -Z (forward)
    x_cam = coords_local[0]
    y_cam = -coords_local[2]  # LiDAR Z (up) -> Camera Y (down)
    z_cam = coords_local[1]   # LiDAR Y (north) -> Camera Z (forward)
    print("Camera-relative Z:", z_cam.min(), z_cam.max())

    # --- Equirectangular Projection ---
    r = np.sqrt(x_cam**2 + y_cam**2 + z_cam**2)  # Depth
    theta = np.arctan2(x_cam, z_cam)             # Azimuth
    phi = np.arccos(-y_cam / r)                  # flip Zenith
    
    # Normalized coordinates [0,1] range
    u_norm = 0.5 * (theta/np.pi + 1)
    v_norm = phi/np.pi

    # Convert to pixel coordinates using precise scaling
    u_idx = np.floor(u_norm * (width - 1)).astype(np.int32)
    v_idx = np.floor(v_norm * (height - 1)).astype(np.int32)

    # Ensure indices are within bounds
    u_idx = np.clip(u_idx, 0, width - 1)
    v_idx = np.clip(v_idx, 0, height - 1)

    # --- Rasterization ---
    rgb_raster = np.full((height, width, 3), nodata_int, dtype=np.uint8)
    z_raster = np.full((height, width), nodata_float, dtype=np.float32)
    depth_raster = np.full((height, width), nodata_float, dtype=np.float32)
    class_raster = np.full((height, width), nodata_int, dtype=np.uint8)
    intensity_raster = np.full((height, width), nodata_int, dtype=np.uint8)

    # Sort points by depth (closest first)
    sort_idx = np.argsort(r)
    u_idx = u_idx[sort_idx]
    v_idx = v_idx[sort_idx]
    r = r[sort_idx]
    rgb = rgb[sort_idx]
    z = z[sort_idx]
    classification = classification[sort_idx]
    intensity = intensity[sort_idx]

    # Vectorized depth test
    valid = (u_idx >= 0) & (u_idx < width) & (v_idx >= 0) & (v_idx < height)
    u_valid = u_idx[valid]
    v_valid = v_idx[valid]

    # Update only if closer than existing depth
    mask = r[valid] < depth_raster[v_valid, u_valid]
    depth_raster[v_valid[mask], u_valid[mask]] = r[valid][mask]
    rgb_raster[v_valid[mask], u_valid[mask]] = (rgb[valid][mask]).astype(np.uint8)
    z_raster[v_valid[mask], u_valid[mask]] = z[valid][mask] + camera_pos[2]
    class_raster[v_valid[mask], u_valid[mask]] = classification[valid][mask]
    intensity_raster[v_valid[mask], u_valid[mask]] = intensity[valid][mask]

    # # for debugging purpose
    # print('number of valid depth points: ',np.sum(depth_raster != nodata_float))
    # print('number of valid elevation points: ',np.sum(z_raster!=nodata_float))
    # print('number of classification points: ',np.sum(class_raster != nodata_int))
    # print('number of valid rgb points: ',np.sum(np.any(rgb_raster!= nodata_int, axis=-1)))
    # print('number of valid intensity points: ',np.sum(intensity_raster != nodata_int))
    
    return rgb_raster, z_raster, depth_raster, class_raster, intensity_raster


### Reproject trajectory coordinates to match lidar points

In [173]:
transformer = pyproj.Transformer.from_crs("EPSG:7844",  # GDA2020 geographic (lat/lon)
                                          "EPSG:7855",      # MGA Zone 55 (EPSG:7855)
                                          always_xy=True)
lat, lon, elev = [gdf_building_points.iloc[i]['LATITUDE'],gdf_building_points.iloc[i]['LONGITUDE'],gdf_building_points.iloc[i]['LTP_z_m']] # lidar data is in AHD
x_proj, y_proj = transformer.transform(lon, lat)  # lon, lat order
camera_pos_proj = [x_proj, y_proj, elev]
camera_angles=[gdf_building_points.iloc[i]['Heading_deg'], gdf_building_points.iloc[i]['Pitch_deg'], gdf_building_points.iloc[i]['Roll_deg']]

### Densify points (optional depending on performance)

In [174]:
# out_densified_path=las_file_path.split('.')[0]+'_densified.las'

In [175]:
# pipeline_json = {
#     "pipeline": [
#         las_file_path,
#         {
#             "type": "filters.poisson",
#             "depth": 10, # Start with a mid-range depth (this controls resolution)
#         },
#         out_densified_path
#     ]
# }
# pipeline = pdal.Pipeline(json.dumps(pipeline_json))
# pipeline.execute()


### Project the point cloud as rasters
* Note: using the same width/height ratio as panoramas
* reduced resolution to improve sampling of surface points compared to background points

In [185]:
upper_crop=0.25
lower_crop=0.6 # consistent with panorama clipping
width_panorama=11000
height_panorama=5500  # panoramas parameters
downscale_factor=4 # scale factor between panorama and projected lidar rasters
width=int(width_panorama/downscale_factor)
height=int(height_panorama/downscale_factor)
rgb, z, depth, classification, intensity = project_las_to_equirectangular(input_las=las_file_path, camera_pos=camera_pos_proj,
                                                               camera_angles=camera_angles, width=width, height=height)

RGB min/max: [0. 0. 0.] [255. 255. 255.]
Camera-relative Z: -10.835831662816577 11.396926281034848


### Interpolate gaps on elevation and intensity rasters

In [186]:
def fill_small_nans(arr, max_hole_size=10, nodata_value=9999):
    """
    Fills small nodata regions using local interpolation from surrounding valid pixels.
    
    Parameters:
        arr: 2D numpy array with nodata values
        max_hole_size: Maximum size (in pixels) of nodata regions to fill
        nodata_value: The value representing nodata (default: 9999)
        
    Returns:
        Array with small nodata regions filled, large ones preserved
    """
    print(f"Initial nodata count: {np.sum(arr == nodata_value)}")
    
    # Create mask of nodata regions
    nodata_mask = (arr == nodata_value)
    
    # Label connected nodata regions
    labeled, num_features = ndimage.label(nodata_mask)
    
    # Measure size of each nodata region
    sizes = ndimage.sum(nodata_mask, labeled, range(num_features + 1))
    
    # Create output array
    filled = arr.copy()
    
    # Compute global distance transform once (from all valid pixels)
    distances, indices = ndimage.distance_transform_edt(
        nodata_mask,  # Important: input is the nodata mask
        return_indices=True
    )
    
    # Process each nodata region
    for i in range(1, num_features + 1):
        region_mask = (labeled == i)
        region_size = np.sum(region_mask)
        
        if region_size <= max_hole_size:
            # Fill using precomputed nearest valid pixels
            filled[region_mask] = arr[
                indices[0][region_mask], 
                indices[1][region_mask]
            ]
    print(f"Final nodata count: {np.sum(filled == nodata_value)}")
    return filled


In [187]:
# fill small holes
# z_arr_clipped_filled = fill_small_nans(z_arr_clipped, max_hole_size=5)
z_arr_filled = fill_small_nans(z, max_hole_size=10, nodata_value=9999)
intensity_filled = fill_small_nans(intensity, max_hole_size=10, nodata_value=255)

Initial nodata count: 3742351
Final nodata count: 3737042
Initial nodata count: 3742398
Final nodata count: 3737067


### Upsample to the same resolution as panorama

In [188]:
def resize_preserve_nans(arr, target_height, target_width, nodata_value=9999):
    """
    Resizes an array while preserving NoData regions, preventing artifacts at edges.
    """
    # Create valid mask (1=valid, 0=nodata)
    valid_mask = (arr != nodata_value)
    
    # For interpolation, replace nodata with 0 but we'll mask later
    # arr_filled = np.where(valid_mask, arr, 0)
    
    # Compute scale factors
    scale_y = arr.shape[0] / target_height
    scale_x = arr.shape[1] / target_width

    # Create coordinate grids for interpolation
    y_idx, x_idx = np.meshgrid(np.linspace(0.5, arr.shape[0]-0.5, target_height),
                               np.linspace(0.5, arr.shape[1]-0.5, target_width),
                               indexing='ij')
    coords = np.array([y_idx.ravel(), x_idx.ravel()])

    # Resize mask using nearest-neighbor to keep sharp edges
    resized_mask = resize(valid_mask.astype(float),
                         (target_height, target_width),
                         order=0,  # Nearest-neighbor
                         anti_aliasing=False) > 0.5

    # Create distance-to-edge map to identify border regions
    from scipy.ndimage import distance_transform_edt
    dist_to_nodata = distance_transform_edt(valid_mask)
    edge_zone = dist_to_nodata <= 1  # Pixels adjacent to nodata

    # Interpolate main data
    # resized_data = map_coordinates(arr_filled, coords, order=1, cval=0)
    resized_data = map_coordinates(arr, coords, order=1, cval=nodata_value)
    resized_data = resized_data.reshape((target_height, target_width))

    # For edge pixels, use nearest-neighbor to prevent bleeding
    if np.any(edge_zone):
        # edge_data = map_coordinates(arr_filled, coords, order=0, cval=0)
        edge_data = map_coordinates(arr, coords, order=0, cval=nodata_value)
        edge_data = edge_data.reshape((target_height, target_width))
        
        # Find where original edge pixels map to in output
        edge_coverage = map_coordinates(edge_zone.astype(float), coords, order=1)
        edge_coverage = edge_coverage.reshape((target_height, target_width)) > 0.1
        
        # Use nearest-neighbor result for edge-affected areas
        resized_data = np.where(edge_coverage, edge_data, resized_data)

    # Restore NoData values using the resized mask
    resized_data[~resized_mask] = nodata_value
    
    return resized_data

In [189]:
# # resize while preserving remaining NaNs
# new_width = z_arr_clipped_filled.shape[0]*downscale_factor
# new_height = z_arr_clipped_filled.shape[1]*downscale_factor
# z_arr_filled_resampled=resize_preserve_nans(z_arr_clipped_filled,new_width,new_height)

z_filled_resampled=resize_preserve_nans(z_arr_filled,height_panorama, width_panorama, nodata_value=9999)
intensity_filled_resampled=resize_preserve_nans(intensity_filled, height_panorama, width_panorama, nodata_value=255)

### Clip projected rasters to the same region as panoramas
* only elevation and intensity rasters were upsampled for use in next steps

In [193]:
z_processed=z_filled_resampled[house_loc_left:house_loc_right, 
                               int(round(upper_crop*height_panorama)):int(round(lower_crop*height_panorama))]
intensity_processed=intensity_filled_resampled[house_loc_left:house_loc_right, 
                                               int(round(upper_crop*height_panorama)):int(round(lower_crop*height_panorama))]

* other rasters are unprocessed and low resolution for visual check only

In [194]:
min_row, max_row=int(round(house_loc_left/downscale_factor)), int(round(house_loc_right/downscale_factor))
min_col, max_col=int(round(upper_crop*height)), int(round(lower_crop*height))

rgb_arr_clipped=rgb[min_row:max_row, min_col:max_col,:]
class_arr_clipped=classification[min_row:max_row, min_col:max_col]

### Save rasters

In [195]:
out_path_rgb=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_rgb.tif'))
out_path_elevation=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_elevation_resampled.tif'))
out_path_intensity=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_intensity_resampled.tif'))
out_path_classification=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_classification.tif'))

In [196]:
imsave(out_path_rgb, rgb_arr_clipped)
imsave(out_path_classification, class_arr_clipped)
imsave(out_path_elevation, z_processed)
imsave(out_path_intensity, intensity_processed)

## Batch apply to all buildings with nearby panoramas

In [ ]:
for i in range(len(gdf_building_points)):
    building_ufi=gdf_building_points.iloc[i]['UFI']
    house_loc_left, house_loc_right = int(gdf_building_points.iloc[i]['house_loc_left']), int(gdf_building_points.iloc[i]['house_loc_right'])
    try:
        # find las by UFI
        las_file_path=glob.glob(las_files_folder+'*'+'_UFI_'+str(building_ufi)+'.las')[0]

        # get viewpoint metadata
        lat, lon, elev = [gdf_building_points.iloc[i]['LATITUDE'],gdf_building_points.iloc[i]['LONGITUDE'],gdf_building_points.iloc[i]['LTP_z_m']] # lidar data is in AHD
        x_proj, y_proj = transformer.transform(lon, lat)  # lon, lat order
        camera_pos_proj = [x_proj, y_proj, elev]
        camera_angles=[gdf_building_points.iloc[i]['Heading_deg'], gdf_building_points.iloc[i]['Pitch_deg'], gdf_building_points.iloc[i]['Roll_deg']]
        
        # project as rasters
        rgb, z, depth, classification, intensity = project_las_to_equirectangular(input_las=las_file_path, camera_pos=camera_pos_proj,
                                                                    camera_angles=camera_angles, width=width, height=height)
        
        # fill small holes
        z_arr_filled = fill_small_nans(z, max_hole_size=10, nodata_value=9999)
        intensity_filled = fill_small_nans(intensity, max_hole_size=10, nodata_value=255)

        # upsample
        z_filled_resampled=resize_preserve_nans(z_arr_filled,height_panorama, width_panorama, nodata_value=9999)
        intensity_filled_resampled=resize_preserve_nans(intensity_filled, height_panorama, width_panorama, nodata_value=255)

        # crop
        z_processed=z_filled_resampled[house_loc_left:house_loc_right, 
                                    int(round(upper_crop*height_panorama)):int(round(lower_crop*height_panorama))]
        intensity_processed=intensity_filled_resampled[house_loc_left:house_loc_right, 
                                                    int(round(upper_crop*height_panorama)):int(round(lower_crop*height_panorama))]
        
        min_row, max_row=int(round(house_loc_left/downscale_factor)), int(round(house_loc_right/downscale_factor))
        min_col, max_col=int(round(upper_crop*height)), int(round(lower_crop*height))
        rgb_arr_clipped=rgb[min_row:max_row, min_col:max_col,:]
        class_arr_clipped=classification[min_row:max_row, min_col:max_col]

        #save
        out_path_rgb=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_rgb.tif'))
        out_path_elevation=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_elevation_resampled.tif'))
        out_path_intensity=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_intensity_resampled.tif'))
        out_path_classification=os.path.join(out_folder,os.path.basename(las_file_path).replace('.las','_classification.tif'))

        imsave(out_path_rgb, rgb_arr_clipped)
        imsave(out_path_classification, class_arr_clipped)
        imsave(out_path_elevation, z_processed)
        imsave(out_path_intensity, intensity_processed)

    except Exception as e:
        print(e)